# Interactive image swipe comparison (RGB/HSV/Channels/B-R)

This notebook provides an interactive viewer to compare two images using a **swipe overlay** (horizontal or vertical).
It supports:
- Cropping the same ROI from both images
- Viewing modes: RGB, HSV (visualized), Saturation, R, G, B, and B/R ratio
- Saving the current comparison figure as a PNG (Optional)

In [ ]:
import os
import numpy as np
import imageio.v3 as iio
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import ipywidgets as widgets
from IPython.display import display

## Image helpers

### Goals
- Ensure every image is treated as **RGB** for consistent downstream processing.
- Provide a safe crop function that clamps requested crop parameters to image bounds.

Notes:
- Some readers may return grayscale arrays shaped `(H, W)`; these are expanded to `(H, W, 3)`.

In [ ]:
def ensure_rgb(img):
    """
    Ensure the image is RGB-like with shape (H, W, 3).

    - If grayscale (H, W), stack it into 3 channels.
    - If multi-channel, keep first 3 channels (drop alpha or extras).
    """
    if img.ndim == 2:
        return np.stack([img, img, img], axis=-1)

    if img.ndim == 3 and img.shape[2] >= 3:
        return img[..., :3]

    raise ValueError("Unsupported image shape")


def crop(img, top, left, height, width):
    """
    Crop by (top, left, height, width), with boundary checks.

    - If height/width <= 0, crop to the image edge.
    - Clamp top/left and bottom/right to valid image coordinates.
    """
    h, w = img.shape[:2]

    t = max(0, min(top, h - 1))
    l = max(0, min(left, w - 1))

    if height <= 0:
        height = h - t
    if width <= 0:
        width = w - l

    b = max(0, min(t + height, h))
    r = max(0, min(l + width,  w))

    return img[t:b, l:r]

## Color space utilities (NumPy only)

This section implements **vectorized** RGB↔HSV conversion in NumPy.

Why:
- Keeps everything inside NumPy arrays (no per-pixel Python loops).
- Enables derived views such as Saturation or a B/R ratio image.

Conventions:
- Input RGB is `uint8` in [0..255]
- HSV/RGB float outputs are normalized to [0..1]

In [ ]:
def rgb_to_hsv_np(rgb_uint8):
    """Vectorized RGB->HSV. Input uint8 [0..255], output float HSV in [0..1]."""
    rgb = rgb_uint8.astype(np.float32) / 255.0
    r, g, b = rgb[..., 0], rgb[..., 1], rgb[..., 2]

    cmax = np.max(rgb, axis=-1)
    cmin = np.min(rgb, axis=-1)
    delta = cmax - cmin + 1e-12  # avoid division by zero

    # Hue in [0..1]
    hue = np.zeros_like(cmax)
    mask = delta > 1e-8

    r_is_max = (cmax == r) & mask
    g_is_max = (cmax == g) & mask
    b_is_max = (cmax == b) & mask

    hue[r_is_max] = ((g - b)[r_is_max] / delta[r_is_max]) % 6.0
    hue[g_is_max] = ((b - r)[g_is_max] / delta[g_is_max]) + 2.0
    hue[b_is_max] = ((r - g)[b_is_max] / delta[b_is_max]) + 4.0
    hue = hue / 6.0

    # Saturation in [0..1]
    sat = np.zeros_like(cmax)
    nonzero = cmax > 1e-8
    sat[nonzero] = delta[nonzero] / cmax[nonzero]

    # Value in [0..1]
    val = cmax

    return np.stack([hue, sat, val], axis=-1)


def hsv_to_rgb_np(hsv):
    """Vectorized HSV->RGB. Input HSV float in [0..1], output RGB float in [0..1]."""
    h, s, v = hsv[..., 0], hsv[..., 1], hsv[..., 2]
    h6 = h * 6.0

    i = np.floor(h6).astype(np.int32) % 6
    f = h6 - np.floor(h6)

    p = v * (1.0 - s)
    q = v * (1.0 - f * s)
    t = v * (1.0 - (1.0 - f) * s)

    r = np.zeros_like(v)
    g = np.zeros_like(v)
    b = np.zeros_like(v)

    idx = i == 0
    r[idx], g[idx], b[idx] = v[idx], t[idx], p[idx]
    idx = i == 1
    r[idx], g[idx], b[idx] = q[idx], v[idx], p[idx]
    idx = i == 2
    r[idx], g[idx], b[idx] = p[idx], v[idx], t[idx]
    idx = i == 3
    r[idx], g[idx], b[idx] = p[idx], q[idx], v[idx]
    idx = i == 4
    r[idx], g[idx], b[idx] = t[idx], p[idx], v[idx]
    idx = i == 5
    r[idx], g[idx], b[idx] = v[idx], p[idx], q[idx]

    return np.clip(np.stack([r, g, b], axis=-1), 0.0, 1.0)

## Display preparation (view modes)

`prepare_view(img, mode)` converts an image crop into something directly displayable with `imshow`:

- `RGB`: normalized RGB image
- `HSV`: computed HSV, then converted back to RGB for visualization
- `Saturation`: single-channel grayscale (S)
- `R`, `G`, `B`: single-channel grayscale per channel
- `B/R`: ratio image (with percentile clipping to reduce outlier impact)

In [ ]:
def prepare_view(img_uint8, mode):
    """
    Return an array ready for imshow:
    - RGB modes return (H, W, 3) float in [0,1]
    - Single-channel modes return (H, W) float in [0,1]
    """
    rgb = ensure_rgb(img_uint8)

    if mode == "RGB":
        return np.clip(rgb.astype(np.float32) / 255.0, 0.0, 1.0)

    hsv = rgb_to_hsv_np(rgb)

    if mode == "HSV":
        # HSV isn't directly displayable with imshow; convert back to RGB for viewing.
        return hsv_to_rgb_np(hsv)

    if mode == "Saturation":
        return hsv[..., 1]

    if mode == "R":
        return rgb[..., 0].astype(np.float32) / 255.0

    if mode == "G":
        return rgb[..., 1].astype(np.float32) / 255.0

    if mode == "B":
        return rgb[..., 2].astype(np.float32) / 255.0

    if mode == "B/R":
        r = rgb[..., 0].astype(np.float32)
        b = rgb[..., 2].astype(np.float32)
        eps = 1e-6

        ratio = b / (r + eps)

        # Clip robustly to reduce extreme values (e.g., specular highlights).
        finite = np.isfinite(ratio)
        lo, hi = np.percentile(ratio[finite], [1, 99]) if finite.any() else (0.0, 1.0)
        if hi <= lo:
            lo, hi = ratio.min(), ratio.max()

        ratio = np.clip(ratio, lo, hi)
        denom = (hi - lo) if hi > lo else 1.0
        return (ratio - lo) / denom

    raise ValueError("Unknown mode")

## Swipe overlay visualization

`show_swipe(ax, imgA, imgB, pos, orientation)`:
- Plots image A as the base.
- Plots image B on top, but **clips** it with a rectangle controlled by `pos`.
- Supports horizontal swipe (left-to-right) or vertical swipe (top-to-bottom).

This uses Matplotlib clipping with a patch (rectangle) applied to the overlaid image.

In [ ]:
def show_swipe(ax, imgA, imgB, pos, orientation="Horizontal", alpha_top=1.0):
    ax.clear()
    ax.set_axis_off()

    # Ensure same HxW by cropping both to the minimum common size
    h = min(imgA.shape[0], imgB.shape[0])
    w = min(imgA.shape[1], imgB.shape[1])
    A = imgA[:h, :w]
    B = imgB[:h, :w]

    # Base image (A)
    if A.ndim == 2:
        ax.imshow(A, cmap="gray", vmin=0, vmax=1)
    else:
        ax.imshow(A)

    # Clipping rectangle for image B based on slider position
    if orientation == "Horizontal":
        cut = int(w * pos)
        clip_rect = Rectangle((0, 0), cut, h, transform=ax.transData)
    else:
        cut = int(h * pos)
        clip_rect = Rectangle((0, 0), w, cut, transform=ax.transData)

    # Overlaid image (B) clipped to rectangle
    if B.ndim == 2:
        imB = ax.imshow(B, cmap="gray", vmin=0, vmax=1, alpha=alpha_top)
    else:
        imB = ax.imshow(B, alpha=alpha_top)

    imB.set_clip_path(clip_rect)
    ax.figure.canvas.draw_idle()

## Select images and build widgets

- Reads all images from the path with extensions: jpg/jpeg/png
- Provides:
  - Dropdowns to choose Image 1 and Image 2
  - Crop controls (top/left/height/width)
  - Viewing mode toggle
  - Swipe orientation + swipe position slider
  - Alpha slider for the overlaid (top) image

The plot is shown in an `Output()` widget so updates replace the previous plot cleanly (via `clear_output`). [web:21]

In [ ]:
image_folder = "Images"
exts = (".jpg", ".jpeg", ".png")

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(exts)]
if not image_files:
    raise RuntimeError("No images found in input/passed")

pick1 = widgets.Dropdown(options=image_files, description="Image 1")
pick2 = widgets.Dropdown(options=image_files, description="Image 2")

top    = widgets.IntSlider(value=30, min=0, max=2000, step=1, description="Top")
left   = widgets.IntSlider(value=30, min=0, max=2000, step=1, description="Left")
height = widgets.IntSlider(value=930, min=0, max=2160, step=1, description="Height")
width  = widgets.IntSlider(value=930, min=0, max=3840, step=1, description="Width")

mode = widgets.ToggleButtons(
    options=["RGB", "HSV", "Saturation", "R", "G", "B", "B/R"],
    description="Mode"
)

orientation = widgets.ToggleButtons(
    options=["Horizontal", "Vertical"],
    description="Swipe"
)

pos = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description="Position")
alpha_top = widgets.FloatSlider(value=1.0, min=0.2, max=1.0, step=0.05, description="Top α")

refresh_btn = widgets.Button(description="Refresh")
out = widgets.Output()

## Render function + event wiring

`render()`:
1. Clears previous output (so plots don't stack).
2. Reads both images.
3. Applies the same crop parameters to both.
4. Computes display views based on the selected mode.
5. Draws the swipe overlay and saves a PNG snapshot.

Then widget events (`observe`) call `render()` whenever a control changes.

In [ ]:
def render(_=None):
    # Clear the previous figure output so only the latest view is shown
    out.clear_output(wait=True)  # smoother updates in notebooks [web:21]

    f1, f2 = pick1.value, pick2.value
    img1 = iio.imread(os.path.join(image_folder, f1))
    img2 = iio.imread(os.path.join(image_folder, f2))

    # Crop both images using identical ROI
    img1c = crop(img1, top.value, left.value, height.value, width.value)
    img2c = crop(img2, top.value, left.value, height.value, width.value)

    # Prepare selected view (RGB/HSV/Sat/Channels/Ratio)
    v1 = prepare_view(img1c, mode.value)
    v2 = prepare_view(img2c, mode.value)

    # Display swipe comparison
    with out:
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        show_swipe(ax, v1, v2, pos.value, orientation.value, alpha_top=alpha_top.value)
        ax.set_title(f"{mode.value}: {f1} vs {f2}")
        plt.tight_layout()

        # Save snapshot of the current comparison view
        #fig.savefig(f"{f1}_vs_{f2}_{mode.value}.png")
        plt.show()

# Re-render when any control changes
for w in [pick1, pick2, top, left, height, width, mode, orientation, pos, alpha_top]:
    w.observe(render, names="value")

refresh_btn.on_click(render)

## Display UI

This cell lays out the controls and shows the interactive output area, then renders once initially.

In [ ]:
controls = widgets.VBox([
    widgets.HBox([pick1, pick2]),
    widgets.HBox([mode, orientation]),
    widgets.HBox([pos, alpha_top, refresh_btn]),
    widgets.HBox([top, left]),
    widgets.HBox([height, width]),
])

display(controls, out)
render()